In [6]:
# 載入需要的套件
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import schedule

In [7]:
#關通知
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values':{'notifications': 2}}
options.add_experimental_option('prefs', prefs)

#不顯示視窗 覺得視窗礙眼可以用
#options.add_argument("--headless")
#options.add_argument('--disable-gpu') 

# ------ 設定要前往的網址 ------
url = 'https://www.facebook.com'  

# ------ 登入的帳號與密碼 ------
username = 'marksu901104@gmail.com'
password = 'm.s.1104'

# 找社團(這是二手拍連結，並按照最新貼文排序)
url_g = 'https://www.facebook.com/groups/817620721658179?sorting_setting=CHRONOLOGICAL'

#元素定位
poster_class = 'x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xi81zsa x1yc453h'
date_class = 'x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm'
contents_class = 'x1iorvi4 x1pi30zi x1l90r2v x1swvt13'

#文章暫存
temp = ''

In [8]:
#點擊要打開的東西
def click_name(driver,name):
    
    str = "//*[text()='%s']" % name
    more_button = driver.find_elements(By.XPATH,str)
    for i in range(len(more_button)):
        try:
            more_button[i].click()
        except:
            continue

#line 通知連動
def line_notify(content):
    headers = {
            "Authorization": "Bearer " + "09ZRoR5lAVH9PGVaqL7ae0rTXwiUnuRl5SBrsD7RKdE",
            "Content-Type": "application/x-www-form-urlencoded"
        }
    params = {"message": time.strftime("[ %m-%d %H:%M:%S ] ", time.localtime()) + '\n' + "二手拍有您所關注字詞的貼文，貼文如下：" + '\n' + content + '\n' + url_g}

    r = requests.post("https://notify-api.line.me/api/notify",headers=headers, params=params)
    print(r.status_code)  #200

#這個function沒用到 用來輸出成exel的
def out_put(poster,date,contents):
   collect_fb = pd.DataFrame()
      
   df_fb = pd.DataFrame({
                      'person': pd.Series(poster),
                      'date': pd.Series(date),
                      'content': pd.Series(contents),
                     })
   collect_fb = pd.concat([collect_fb, df_fb], ignore_index=True)
   
   writer = pd.ExcelWriter('output1.xlsx', engine='xlsxwriter')
   collect_fb.to_excel(writer, sheet_name='fb')
   writer.close()

#滾動視窗 因為臉書要下滑才會顯示更多   
def rolling(count):
    # 往下滑
    for j in range(count):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(1)
    # 往回滑    
    #driver.execute_script("window.scrollTo(0,-document.body.scrollHeight)")

In [9]:
# ------ 透過Browser Driver 開啟 Chrome ------
driver = webdriver.Chrome(options=options)  

# 最大化視窗 
driver.maximize_window()     

# ------ 前往該網址 ------
driver.get(url)    

# ------ 輸入賬號密碼 ------
driver.find_element(By.ID,'email').send_keys(username)
driver.find_element(By.ID,'pass').send_keys(password)        
driver.find_element(By.NAME,'login').click()
time.sleep(1)

# 找社團
driver.get(url_g)
time.sleep(1)

In [10]:
driver.refresh()
time.sleep(1)
rolling(1)
time.sleep(1)
#這個 '顯示更多' 變成 '查看更多'了
click_name(driver,'查看更多')
time.sleep(1)

contents = []
date = []
poster = []
articles = []

htmltext = driver.page_source
time.sleep(1)
soup = BeautifulSoup(htmltext, 'html.parser')

# 抓取文章
articles = soup.find_all(class_ ='x9f619 x1n2onr6 x1ja2u2z x2bj2ny x1qpq9i9 xdney7k xu5ydu1 xt3gfkd xh8yej3 x6ikm8r x10wlt62 xquyuld')

# 丟掉不是文章的
i=0
while True:
   if articles[i].find(class_ = contents_class) == None or articles[i].find(class_ = poster_class) == None or articles[i].find(class_ = date_class) == None:
      articles.pop(i)
      i=i-1
      if i+1 == len(articles):
         break
   i=i+1
      
for a in articles:
   poster.append(a.find(class_ = poster_class).text)
   date.append(a.find(class_ = date_class).text)
   contents.append(a.find(class_ = contents_class).text)
   
#driver.close()

#這邊是只判斷第一篇文章 print過的就不print
#有包含 '椅' 的內容會連動line通知
global temp 
if contents[0] != temp:
   if '椅' in contents[0]:
      line_notify(contents[0])
         
   temp = contents[0]   
   print(time.strftime("[ %m-%d %H:%M:%S ] ", time.localtime()) + '\n' + contents[0])
                  
#out_put(poster,date,contents)


[ 07-10 23:37:11 ] 
《贈》營隊剩下的宵夜馬露蓮 嫩仙草+芋圓 10碗工四大廳的桌上工四大廳的桌上工四大廳的桌上最後一個拿的幫我留言還有清理垃圾！謝謝！
